# SVM

In [235]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math
train_data=open('titanic_x_y_train.csv')
train_data=pd.read_csv(train_data)
test_data=open('titanic_x_test.csv')
test_data=pd.read_csv(test_data)

In [236]:
train_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.000,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.050,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.000,NaN,S,0
3,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,0
4,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,0


In [237]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,"Davies, Master. John Morgan Jr",male,8.0,1,1,C.A. 33112,36.7500,NaN,S
1,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,17465,25.9292,D17,S
2,3,"Kilgannon, Mr. Thomas J",male,NaN,0,0,36865,7.7375,NaN,Q
3,2,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",female,24.0,2,1,243847,27.0000,NaN,S
4,1,"McGough, Mr. James Robert",male,36.0,0,0,PC 17473,26.2875,E25,S


In [238]:
print(train_data.shape)
print(test_data.shape)

(668, 11)
(223, 10)


In [239]:
train_data=train_data.drop('Name',axis=1)
train_data=train_data.drop('Cabin',axis=1)
train_data=train_data.drop('Ticket',axis=1)
train_data=train_data.dropna()

y_train=train_data['Survived']
x_train=train_data.drop('Survived',axis=1)

#x_train=np.array(x_train)
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,female,29.0,1,0,26.0000,S
2,2,male,39.0,0,0,26.0000,S
3,3,female,29.0,0,4,21.0750,S
4,3,male,25.0,0,0,7.0500,S
5,3,male,34.5,0,0,6.4375,C


In [240]:
x_train = pd.get_dummies(x_train, columns=["Pclass","Sex", "Embarked"])
x_train.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,29.0,1,0,26.0000,0,1,0,1,0,0,0,1
2,39.0,0,0,26.0000,0,1,0,0,1,0,0,1
3,29.0,0,4,21.0750,0,0,1,1,0,0,0,1
4,25.0,0,0,7.0500,0,0,1,0,1,0,0,1
5,34.5,0,0,6.4375,0,0,1,0,1,1,0,0


In [241]:
x_train =np.array(x_train)
x_train=preprocessing.scale(x_train)
x_train= pd.DataFrame(x_train)
x_train['c']=1
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,c
0,-0.048122,0.533229,-0.509394,-0.169449,-0.586704,1.696117,-0.972345,1.294218,-1.294218,-0.482409,-0.186591,0.534844,1
1,0.654336,-0.572497,-0.509394,-0.169449,-0.586704,1.696117,-0.972345,-0.772667,0.772667,-0.482409,-0.186591,0.534844,1
2,-0.048122,-0.572497,3.886182,-0.273830,-0.586704,-0.589582,1.028442,1.294218,-1.294218,-0.482409,-0.186591,0.534844,1
3,-0.329105,-0.572497,-0.509394,-0.571078,-0.586704,-0.589582,1.028442,-0.772667,0.772667,-0.482409,-0.186591,0.534844,1
4,0.338230,-0.572497,-0.509394,-0.584059,-0.586704,-0.589582,1.028442,-0.772667,0.772667,2.072928,-0.186591,-1.869705,1


In [242]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [300]:
def single_gradient(x_train,y_train, learning_rate,m):  
    m_slope = [0 for i in range(x_train.shape[1])]
    C=1
    Lambda=1/C
    M = x_train.shape[0]
    N = x_train.shape[1]
    for j in range(N):
        for i in range(M):
            x = x_train[i]
            y = y_train[i]
            y = y = np.where(y == 0, -1, 1)
            a=np.dot(m,x)
            if y*a>=1:
                m_slope[j] += 2*Lambda*m[j]/M
            else:
                m_slope[j] += -y*x[j]/M + 2*Lambda*m[j]/M
    m_slope=np.array(m_slope)
    new_m=m-m_slope*learning_rate   
    return new_m
            
def gd(x_train,y_train, learning_rate, num_iterations):
    m = [0 for i in range(x_train.shape[1])]
    for i in range(num_iterations):
        m = single_gradient(x_train,y_train,learning_rate,m)
        #print(i, " Cost: ", cost(x_train,y_train,m))
    return m

def cost(x_train,y_train, m):
    total_cost = 0
    C=1
    M = x_train.shape[0]
    for i in range(M):
        x = x_train[i]
        y = y_train[i]
        a = np.dot(m,x)
        sum1=0
        for i in m:
            sum1+=i**2
        list1=[0,1-y*a]
        sort=sorted(list1)
        total_cost += (C*sort[1]+1/2*sum1)/M
    return total_cost

def fit(x_train,y_train):
    learning_rate =0.001
    num_iterations = 100
    m = gd(x_train,y_train, learning_rate, num_iterations)
    return m

In [301]:
fit(x_train,y_train)

array([-0.00859998, -0.00094902,  0.00587836,  0.02233412,  0.02631742,
        0.00809771, -0.03007056,  0.04636014, -0.04636014,  0.01572487,
       -0.00307954, -0.01346176, -0.01373466])

In [302]:
def predict(x_test):
    m=fit(x_train,y_train)
    y_predict=[]
    for x in x_test:
        if np.dot(m,x)>0:
            y_predicted=1
        else:
            y_predicted=0
        y_predict.append(y_predicted)
        
    return np.array(y_predict)

In [303]:
def score(y_predict,y_test):
    diff=y_predict-y_test
    sum1=0
    for i in diff:
        if i!=0:
            sum1+=1
    accuracy=1-(sum1/len(y_test))
    return accuracy

In [304]:
y_predict=predict(x_train)

In [305]:
score(y_predict,y_train)

0.7607476635514019